In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pytorch(Scratch):-

In [2]:
import torch 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,sampler

In [3]:
device = torch.cuda.is_available()
'GPU is available!!!' if device else 'GPU isn\'t available'

'GPU is available!!!'

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

train = datasets.CIFAR10('data/CIFAR',train = True,transform=transform,download = True)
test = datasets.CIFAR10('data/CIFAR',train = False,transform=transform,download = True)

valid_size = 0.2
num_train = len(train)
idx = list(range(num_train))
np.random.shuffle(idx)
valid_idx = idx[:int(num_train*valid_size)]
train_idx = idx[int(num_train*valid_size):]

trainsampler = sampler.RandomSampler(train_idx)
validsampler = sampler.RandomSampler(valid_idx)

trainloader = DataLoader(train,sampler=trainsampler,batch_size=32)
validloader = DataLoader(train,sampler=validsampler,batch_size=32)
testloader = DataLoader(test,batch_size=32)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
from torch import nn,optim
import torch.nn.functional as F

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x
    
model = Net()
print(model)

if device:
    model.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [24]:
epochs = 20
min_valid_loss = np.Inf

for i in range(1,epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for data,label in trainloader:
        if device:
            data,label = data.cuda(),label.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    
    model.eval()
    for data,label in validloader:
        if device:
            data,label = data.cuda(),label.cuda()
        output = model(data)
        loss = criterion(output,label)
        valid_loss += loss.item()*data.size(0)
    
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
    print('Epoch {}/20 Training Loss is: {:.6f} Validation Loss is:{:.6f}'.format(
        i,train_loss,valid_loss
    ))
    if min_valid_loss > valid_loss:
        print('Validation loss decreased({:.6f}--->{:.6f}). Saving Model...'.format(min_valid_loss,valid_loss))
        torch.save(model.state_dict(),'mnist.pt')
        min_valid_loss = valid_loss

Epoch 1/20 Training Loss is: 1.871178 Validation Loss is:1.643556
Validation loss decreased(inf--->1.643556). Saving Model...
Epoch 2/20 Training Loss is: 1.624241 Validation Loss is:1.523650
Validation loss decreased(1.643556--->1.523650). Saving Model...
Epoch 3/20 Training Loss is: 1.507047 Validation Loss is:1.383758
Validation loss decreased(1.523650--->1.383758). Saving Model...
Epoch 4/20 Training Loss is: 1.412758 Validation Loss is:1.314863
Validation loss decreased(1.383758--->1.314863). Saving Model...
Epoch 5/20 Training Loss is: 1.341326 Validation Loss is:1.228914
Validation loss decreased(1.314863--->1.228914). Saving Model...
Epoch 6/20 Training Loss is: 1.273918 Validation Loss is:1.158536
Validation loss decreased(1.228914--->1.158536). Saving Model...
Epoch 7/20 Training Loss is: 1.213177 Validation Loss is:1.101805
Validation loss decreased(1.158536--->1.101805). Saving Model...
Epoch 8/20 Training Loss is: 1.157912 Validation Loss is:1.029083
Validation loss decrea

In [25]:
model.load_state_dict(torch.load('mnist.pt'))

<All keys matched successfully>

In [26]:
test_loss = 0.0
correct = 0
total = 0
model.eval()
for data,label in testloader:
    if device:
        data,label = data.cuda(),label.cuda()
    output = model(data)
    loss = criterion(output,label)
    test_loss += loss.item()*data.size(0)
    for o,l in zip(output,label):
        if torch.argmax(o) == l:
            correct += 1
        total += 1
print('Test Loss is: {}'.format(test_loss/len(test)))
print('Accuracy: {}/{}'.format(correct,total))

Test Loss is: 0.8288020639419555
Accuracy: 7080/10000
